In [1]:
import os
import math
import numpy as np
import pandas as pd

In [2]:

df = pd.read_csv('https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points.csv', encoding = "ISO-8859-1")


/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (8,9,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
serve_direction = {'4':'out wide', 
                   '5':'body',
                   '6':'down the T', 
                   '0':'unknown'
                  }
serve_faults = {'n':'net (anything that goes into the net, including net cords that are not lets)',
                'w':'wide (in either direction)',
                'd':'deep',
                'x':'both wide and deep',
                'g':'foot faults',
                'e':'unknown',
                '!':'shank',
                'V':'time violation',
                'c':'let',
                '+':'serve and volley',
                '*':'Ace',
                '#':'forced error',
                '@':'unforced error'
              }
shot_type = {
    'f':'forehand groundstroke (excluding slices, chips, etc.)',
    'b':'backhand groundstroke (excluding slices, chips, etc.)',
    'r':'forehand slice (including defensive chips, but not drop shots)',
    's':'backhand slice (including defensive chips, but not drop shots)',
    'v':'forehand volley',
    'z':'backhand volley',
    'o':'standard overhead/smash',
    'p':'"backhand" overhead/smash',
    'u':'forehand drop shot',
    'y':'backhand drop shot',
    'l':'forehand lob',
    'm':'backhand lob',
    'h':'forehand half-volley',
    'i':'backhand half-volley',
    'j':'forehand swinging volley',
    'k':'backhand swinging volley',
    't':'all trick shots, including behind-the-back, between-the-legs, and "tweeners."',
    'q':'any unknown shot',
    '*':'winner'

}
shot_direction = {
    '1':'to a right-hander''s forehand side or left-hander''s backhand side',
    '2':'down the middle of the court',
    '3':'to a right-hander''s backhand side or left-hander''s forehand side',
    '0':'unknown'
}
shot_depth = {
    '7':'within the service boxes',
    '8':'behind the service line, but closer to the service line than the baseline',
    '9':'closer to the baseline than the service line'
}
court_position = {
    '^':'Stop volleys/drop volleys',
    ';':'hits the net cord',
    '-':'shot took place at the net',
    '=':'shot took place at the baseline',
    '+':'approach'
}